# Docker for Flywheel

Docker is a program that allows you to create "standard units of software that packages up code and all its dependencies so the application runs quickly and reliably from one computing environment to another."  In other words, you set up a self contained environment with any OS, any specific software you want at any specific version, and any other custom settings you may wish to include, and Docker packages all these settings into something called an "image".  You can then run this image identically on any machine that also has Docker. When an image is run, it creates a self-contained environment called a “container”. The image is basically a piece of software stored on your computer, while the container is the object that’s created when you run that software.  No more worrying about versions, dependencies and operating systems!  In a way, you can think of this as a super lightweight virtual machine that can run on any computer that also has Docker.  

To ensure that our gears will run anywhere, we use Docker to create images, which we then run our gears on.  Each gear may have a custom Docker image, depending on the system or software requirements of that gear. As a developer, you will choose and specify exactly what kind of docker image you want to run your gear on.  That will be covered later in this tutorial.  

For now, we only need to install Docker.

### Install Docker

Docker comes in two flavors: Community Edition (CE)  and Enterprise Edition (EE). EE is a paid version of Docker, and it has more support provided from Docker themselves.   CE is a free docker engine, but don’t worry, they can do exactly the same things. Unless your lab is going to get a professional EE docker account, download the CE edition. 

1. Verify your computer meets Docker's system requirements. See Docker's documentation:
    * [Mac](https://docs.docker.com/docker-for-mac/install/#what-to-know-before-you-install)
    * [Linux](https://docs.docker.com/install/linux/docker-ce/centos/#install-docker-engine---community)
    * [Windows](https://docs.docker.com/docker-for-windows/install/#system-requirements)
2. Follow Docker's instructions for creating an account and installing the app:
    1. [Create a Docker account and sign in](https://hub.docker.com/signup)
    2. Install Docker:
        * [Mac](https://hub.docker.com/editions/community/docker-ce-desktop-mac)
        * [Linux](https://docs.docker.com/install/linux/docker-ce/ubuntu/)
        * [Windows](https://docs.docker.com/docker-for-windows/install/#install-docker-desktop-on-windows)

#### How do I know I did this step correctly?

Open a new terminal window and type docker.  A long list of usage options should appear on your screen, starting with:  
```
Usage: docker [OPTIONS] COMMAND
A self-sufficient runtime for containers
```
### Logging in to docker
Before you can fully utilize docker, and docker hub (essentially a repository for any docker images you make, so they can be accessed anywhere from any computer), you must first create a docker hub account.  To do this, visit dockerhub, and clicking "Sign up for dockerhub".  This will prompt you to generate a username and password.  Your username will be associated with all your docker images, so pick something easy to spell and remember.  

Once you've generated your account, open a terminal on your machine and link your account to docker by typing the following:

`docker login`
 You will then be prompted for your docker username and your docker password.  

Once login is successful, you'll be able to push repos to your docker hub account, which will be discussed later in this tutorial.  

## Getting started with Docker

### Pulling and running images
To start, lets pull and run an image so we can play around with it:

`docker pull python:buster`

This will download the latest python image from docker with a base OS of debian buster.  We can then run the image to get the python interpreter.

``` 
> docker run -it python:buster                              
Python 3.8.5 (default, Jul 22 2020, 12:28:11) 
[GCC 8.3.0] on linux
Type "help", "copyright", "credits" or "license" for more information.
>>> 
```

This tells docker to run the image `python:buster`, it also tells docker to run it interactively (-i), and to allocate a pseudo-TTY (-t).  The meaning of these flags are not important, just know that you need to supply `-it` whenever you want to run an image interactively, without it the image runs and exits normally without any interaction.

We can also run a manual command using this format:
```
docker run -it python:buster /bin/bash               
root@8a9607e0e927:/# python
Python 3.8.5 (default, Jul 22 2020, 12:28:11) 
[GCC 8.3.0] on linux
Type "help", "copyright", "credits" or "license" for more information.
>>> 
```
We are again running the python:buster image interactively, but now we run the command `/bin/bash` which gives us an interactive bash shell.  We can then run python from this bash shell.

Images can be searched for on [Docker Hub](https://hub.docker.com/)

### Creating a docker image

There are two main ways of creating a docker image, using a Dockerfile, or pulling an existing image, and modifying it with `docker commit`.  A Dockerfile provides a list of instructions to run in order to create an image, which is usually more portable and reproducible than a modified image.  At Flywheel, we exclusively publish images built using a Dockerfile, however modifying images can be helpful for debugging creating a docker image, so I will go over both methods here.

#### Dockerfile
So what is a Dockerfile.  Let's create a basic Dockerfile to run a python script.

```Dockerfile
# Start with python on debian as base
FROM python:buster           

# Set the environmental variable FLYWHEEL and change to that directory
ENV FLYWHEEL /flywheel/v0      
WORKDIR ${FLYWHEEL}

# Copy the run.py script into the container
COPY run.py ${FLYWHEEL}/run.py


# Make the run.py script executable
RUN chmod a+x ${FLYWHEEL}/run.py

# Set the entrypoint to be the run.py script
ENTRYPOINT ["/flywheel/v0/run.py"]
```

Then lets make a simple python script `run.py`:

```python
#!/usr/bin/env python
print("Hello, World")
```

Now our directory should look like the following:
```
.
├── Dockerfile
└── run.py

0 directories, 2 files

```

From within this directory, lets build the image and tag it with `[your_username]/hello_world:0.0.1`:

```
> docker build -t flywheel/hello_world:0.0.1 ./ # build current directory ('./') and apply tag

Sending build context to Docker daemon  13.82kB
Step 1/6 : FROM python:buster
 ---> 3189819ced3e
Step 2/6 : ENV FLYWHEEL /flywheel/v0
 ---> Using cache
 ---> 48d6c04314d2
Step 3/6 : WORKDIR ${FLYWHEEL}
 ---> Using cache
 ---> 0bb3ec823b41
Step 4/6 : COPY run.py ${FLYWHEEL}/run.py
 ---> Using cache
 ---> 95c86d4592cc
Step 5/6 : RUN chmod a+x ${FLYWHEEL}/run.py
 ---> Using cache
 ---> 2aca9ddf3d1d
Step 6/6 : ENTRYPOINT ["/flywheel/v0/run.py"]
 ---> Running in ebed7e38837a
Removing intermediate container ebed7e38837a
 ---> 77c5c9b39daa
Successfully built 77c5c9b39daa
Successfully tagged flywheel/hello_world:0.0.1
```

Then if we run this image, it will execute the python script:

```
docker run flywheel/hello_world:0.0.1  
Hello, World
```

##### Adding to the image
Let's say that we need to install a specific python package such as numpy. We will test installing the package interactively, and then once we get it to work, we'll add it to the Dockerfile and rebuild:

```
 docker run -it python:buster /bin/bash
root@da1413e90e40:/# pip install numpy
Collecting numpy
  Downloading numpy-1.19.1-cp38-cp38-manylinux2010_x86_64.whl (14.5 MB)
     |████████████████████████████████| 14.5 MB 5.5 MB/s 
Installing collected packages: numpy
Successfully installed numpy-1.19.1

```
It looks like this command works as expected in the `python:buster` image, but it is always a good idea to check in the image before adding it to the Dockerfile.  Adding this line to the Dockerfile, we get:
```Dockerfile
# Start with python on debian as base
FROM python:buster           

# Install numpy
RUN pip install numpy

# Set the environmental variable FLYWHEEL and change to that directory
ENV FLYWHEEL /flywheel/v0      
WORKDIR ${FLYWHEEL}
...
```
We can modify our python script to use numpy:

```python
#!/usr/bin/env python
iport numpy
print(f'numpy version {numpy.__version__}')
```

We can then rebuild and run the image:
```
> docker build -t flywheel/hello_world:0.0.2 ./
Sending build context to Docker daemon  15.87kB
Step 1/7 : FROM python:buster
 ---> 3189819ced3e
Step 2/7 : RUN pip install numpy
 ---> Using cache
 ---> 6c631094fdbe
Step 3/7 : ENV FLYWHEEL /flywheel/v0
 ---> Using cache
 ---> 041cbec28c25
Step 4/7 : WORKDIR ${FLYWHEEL}
 ---> Using cache
 ---> 7de893707890
Step 5/7 : COPY run.py ${FLYWHEEL}/run.py
 ---> 20059a394c2f
Step 6/7 : RUN chmod a+x ${FLYWHEEL}/run.py
 ---> Running in ac8144fb1189
Removing intermediate container ac8144fb1189
 ---> 594eda9c848d
Step 7/7 : ENTRYPOINT ["/flywheel/v0/run.py"]
 ---> Running in d1c2448e97b4
Removing intermediate container d1c2448e97b4
 ---> b0bad93de06a
Successfully built b0bad93de06a
Successfully tagged flywheel/hello_world:0.0.2

> docker run flywheel/hello_world:0.0.2 
numpy version 1.19.1
```

In this example, it takes such a short time to build the image, that we probably don't need to test the `pip` command in the image first, but in larger images where many things need to be installed and the building takes a significant amount of time, it is really helpful to test each command before you put it in the Dockerfile.

## Common docker images used for Flywheel Gears

* [neurodebian](https://hub.docker.com/_/neurodebian/): NeuroDebian provides neuroscience research software for Debian, Ubuntu, and other derivatives.
* [python](https://hub.docker.com/_/python/): Base python image, can be on Debian/Ubuntu, Windows Server, or Alpine base
* [flywheel/fsl-base](https://hub.docker.com/r/flywheel/fsl-base): Base image for FSL tools
* [flywheel/matlab-mcr](https://hub.docker.com/r/flywheel/matlab-mcr): MATLAB compiler runtime
* [flywheel/tools](https://hub.docker.com/r/flywheel/tools)


### OS's and package managers
* Debian/Ubuntu based